## spaCy

spaCy — это open-source библиотека для NLP, написанная на Python и Cython. В отличие от NLTK, который широко используется для преподавания и исследований, spaCy фокусируется на предоставлении программного обеспечения для разработки.

Её функционал позволяет решать очень широкий спектр задач: от определения частей речи и выделения именованных сущностей до создания собственных моделей для анализа.

Рассмотрим, как происходит обработка данных в SpaCy.
Загруженный для обработки текст последовательно проходит через различные компоненты обработки и сохраняется как экземпляр объекта Doc
Doc является центральной структурой данных в SpaCy, именно в нём хранятся последовательности слов или, как их ещё называют, токенов. Внутри объекта Doc можно выделить два других типа объекта: Token и Span. Token представляет собой ссылку на отдельные слова документа, а Span – ссылку на последовательность из нескольких слов (их можно создавать самостоятельно)
Ещё одной важной структурой данных является объект Vocab, который хранит набор справочных таблиц, общий для всех документов. Это позволяет экономить память и обеспечивать единый источник информации для всех обрабатываемых документов.
Токены документов связаны с объектом Vocab через хеш, используя который можно получить начальные формы слов или другие лексические атрибуты токенов

Рассмотрим операции, с помощью которых можно обработать текст.

#### 1. Базовые операции

Прежде чем начинать работу с текстом, следует импортировать языковую модель. Для русского языка существует официальная модель от SpaCy, поддерживающая токенизацию (разбиение текста на отдельные токены) и ряд других базовых операций:

In [4]:
from spacy.lang.ru import Russian

После импорта и создания экземпляра языковой модели можно начинать обработку текста. Для этого нужно всего лишь передать текст созданному экземпляру:

In [5]:
nlp = Russian()
doc = nlp("Добрый день, погода солнечная.")

Работа с получившимся объектом Doc очень схожа с работой со списками: можно обращаться к нужному токену по индексу или делать срезы из нескольких токенов. А чтобы получить текст токена или среза, можно использовать атрибут text:

In [10]:
token = doc[0]
print(token.text)

span = doc[3:6]
print(span.text)

Добрый
погода солнечная.


In [ ]:
Для получения дополнительной информации о том, какой тип информации содержится в токене, можно использовать следующие атрибуты:

is_alpha – проверка на то, содержит ли токен только буквенные символы
is_punct – проверка на то, является ли токен знаком пунктуации
like_num – проверка на то, является ли токен числом

In [11]:
print("is_alpha:    ", [token.is_alpha for token in doc])
print("is_punct:    ", [token.is_punct for token in doc])
print("like_num:    ", [token.like_num for token in doc])

is_alpha:     [True, True, False, True, True, False]
is_punct:     [False, False, True, False, False, True]
like_num:     [False, False, False, False, False, False]


Рассмотрим ещё пример, где на экран выводятся все токены, предшествующие точке. Чтобы получить такой результат, при переборе токенов следует делать проверку следующего токена, используя атрибут token.i:

In [12]:
for token in doc:
    if token.i+1 < len(doc):
        next_token = doc[token.i+1]
        if next_token.text == ".":
            print(token.text)

солнечная


#### 2. Операции с синтаксисом

Для более сложных операций по обработке текста используются другие модели. Они специально натренированы для задач, связанных с синтаксисом, выделением именованных сущностей и работы со значениями слов. Например, для английского языка существует 3 официальных модели, различающихся размером. Для русского языка на настоящий момент официальная модель ещё не обучена, однако уже есть модель ru2 из сторонних источников, которая умеет работать с синтаксисом.
Чтобы полностью проиллюстрировать возможности SpaCy, будем использовать модели для английского языка. Давайте установим маленькую модель en_core_web_sm, которая отлично подойдёт для демонстрации возможностей.
С использованием этой модели мы можем для каждого из токенов получить часть речи, роль в предложении и токен, от которого он зависит:

In [13]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("New Apple MacBook set launch tomorrow")

for token in doc:
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    token_head = token.head.text
    print(f"{token_text:<12}{token_pos:<10}" \
          f"{token_dep:<10}{token_head:<12}")

New         PROPN     compound  MacBook     
Apple       PROPN     compound  MacBook     
MacBook     PROPN     compound  launch      
set         NOUN      compound  launch      
launch      NOUN      ROOT      launch      
tomorrow    NOUN      npadvmod  launch      


In [16]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

auxiliary
proper noun


В SpaCy также реализована возможность узнать начальную форму слова для любого из токенов (для местоимений используется -PRON-):

In [19]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("I saw a movie yesterday")
print(' '.join([token.lemma_ for token in doc]))

I see a movie yesterday


#### 3. Выделение именованных сущностей 

Часто для работы с текстом требуется выделить сущности, упомянутые в тексте. Чтобы получить список именованных сущностей в документе, используется атрибут doc.ents, а для получения метки для этой сущности – атрибут ent.label_:

In [20]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for 1$ billion")
for ent in doc.ents:
    print(ent.text, ent.label_)


Apple ORG
U.K. GPE
1$ billion MONEY


Здесь также можно использовать атрибут explain, чтобы узнать расшифровки меток именованных сущностей:

In [21]:
print(spacy.explain("GPE"))

Countries, cities, states


А функция displacy поможет наглядно обозначить списки сущностей прямо в тексте:

In [22]:
from spacy import displacy
displacy.render(doc, style='ent', jupyter=True)

#### 6. Определение семантической близости

Два слова могут быть очень схожи по смыслу, но как измерить их близость? В подобных задачах на помощь могут прийти семантические вектора. Если два слова или многословных выражения похожи, то их вектора будут лежать близко друг к другу.

Посчитать семантическую близость векторов в SpaCy несложно, если языковая модель была обучена для решения таких задач. Результат сильно зависит от размера модели, поэтому для этой задачи возьмём модель побольше:

In [24]:
import spacy

nlp = spacy.load("en_core_web_md")
doc1 = nlp("I like burgers")
doc2 = nlp("I like pizza")
print(doc1.similarity(doc2))

0.9244170731374305


Значение может колебаться от нуля до единицы: чем ближе к единице, тем больше схожесть. В примере выше мы сравнивали два документа, однако точно так же можно сравнивать отдельные токены и срезы.

Оценка семантической близости может быть полезна при решении множества задач. Например, с её помощью можно настроить рекомендательную систему, чтобы она предлагала пользователю похожие тексты на основе уже прочитанных.

Важно помнить, что семантическая близость очень субъективна и всегда зависит от контекста задачи. Например, фразы «я люблю собак» и «я ненавижу собак» похожи, поскольку обе выражают мнение о собаках, но в то же время сильно различаются по настроению. В некоторых случаях придётся дополнительное обучить языковые модели, чтобы результаты коррелировали с контекстом вашей задачи.